##### Copyright 2025 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Search re-ranking using Gemini embeddings

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Search_reranking_using_embeddings.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/>

This notebook demonstrates the use of embeddings to re-rank search results. This walkthrough will focus on the following objectives:



1.   Setting up your development environment and API access to use Gemini.
2.   Using Gemini's function calling support to access the Wikipedia API.
3.   Embedding content via Gemini API.
4.   Re-ranking the search results.


This is how you will implement search re-ranking:


1.   The user will make a search query.
2.   You will use Wikipedia API to return the relevant search results.
3.   The search results will be embedded and their relevance will be evaluated by calculating distance metrics like cosine similarity.
4.   The most relevant search result will be returned as the final answer.

> The non-source code materials in this notebook are licensed under Creative Commons - Attribution-ShareAlike CC-BY-SA 4.0, https://creativecommons.org/licenses/by-sa/4.0/legalcode.

## Setup


In [2]:
%pip install -q -U "google-genai>=1.0.0"
%pip install -q wikipedia

  Preparing metadata (setup.py) ... done


Note: The [`wikipedia` package](https://pypi.org/project/wikipedia/) notes that it was "designed for ease of use and simplicity, not for advanced use", and that production or heavy use should instead "use [Pywikipediabot](http://www.mediawiki.org/wiki/Manual:Pywikipediabot) or one of the other more advanced [Python MediaWiki API wrappers](http://en.wikipedia.org/wiki/Wikipedia:Creating_a_bot#Python)".

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [3]:
from google.colab import userdata
from google import genai
API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=API_KEY)

In [4]:
# Define the model to be used in this notebook
# Note: experimental models have more limited quota for API calls.

MODEL_ID="gemini-2.0-flash" # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-pro-exp-03-25"] {"allow-input":true, isTemplate: true}
model=f"models/{MODEL_ID}"

## Function Calling with the Gemini API

As stated earlier, this tutorial uses Gemini's function calling support to access the Wikipedia API. Please refer to the [docs](https://ai.google.dev/docs/function_calling) to learn more about function calling.

### Step 1: Define the Search Function

To cater to the search engine needs, you will design this function in the following way:


*   For each search query, the search engine will use the `wikipedia.search` method to get relevant topics.
*   From the relevant topics, the engine will choose `n_topics(int)` top candidates and will use `gemini-2.0-flash` to extract relevant information from the page.
*   The engine will avoid duplicate entries by maintaining a search history.


In [5]:
from typing import List
import textwrap

import wikipedia
from wikipedia.exceptions import DisambiguationError, PageError

# This is the actual function that would be called based on the model's suggestion
# Define the function with type hints and docstring
def wikipedia_search(search_queries: List[str]) -> List[str]:
  """Search wikipedia for each query and summarize relevant docs.
    Args:
        search_queries: The user query to search wikipedia.

    Returns:
        A list of relevant information from wikipedia based on the query.
  """
  n_topics=3
  search_history = set() # tracking search history
  search_urls = []

  summary_results = []

  for query in search_queries:
    print(f'Searching for "{query}"')
    search_terms = wikipedia.search(query)

    print(f"Related search terms: {search_terms[:n_topics]}")
    for search_term in search_terms[:n_topics]: # select first `n_topics` candidates
      if search_term in search_history: # check if the topic is already covered
        continue

      print(f'Fetching page: "{search_term}"')
      search_history.add(search_term) # add to search history

      try:
        # extract the relevant data by using `gemini-2.0-flash` model
        page = wikipedia.page(search_term, auto_suggest=False)
        url = page.url
        print(f"Information Source: {url}")
        search_urls.append(url)
        page = page.content

        response = client.models.generate_content(
            model=model,
            contents=textwrap.dedent(
                f"""
                Extract relevant information
                about user's query: {query}
                From this source:

                {page}

                Note: Do not summarize. Only Extract and return the relevant information
                """
            ),
        )

        urls = [url]

        if response.candidates[0].citation_metadata:
          extra_citations = response.candidates[0].citation_metadata.citation_sources
          extra_urls = [source.url for source in extra_citations]
          urls.extend(extra_urls)
          search_urls.extend(extra_urls)
          print("Additional citations:", response.candidates[0].citation_metadata.citation_sources)
        try:
          text = response.text
        except ValueError:
          pass
        else:
          summary_results.append(text + "\n\nBased on:\n  " + ',\n  '.join(urls))

      except DisambiguationError:
        print(f"""Results when searching for "{search_term}" (originally for "{query}")
        were ambiguous, hence skipping""")
        continue

      except PageError:
        print(f'{search_term} did not match with any page id, hence skipping.')
        continue

      except:
        print(f'{search_term} did not match with any page id, hence skipping.')
        continue

  print(f"Information Sources:")
  for url in search_urls:
    print('    ', url)

  return summary_results

In [6]:
example = wikipedia_search(["What are LLMs?"])

Searching for "What are LLMs?"
Related search terms: ['Large language model', 'Vibe coding', 'DeepSeek']
Fetching page: "Large language model"
Information Source: https://en.wikipedia.org/wiki/Large_language_model
Large language model did not match with any page id, hence skipping.
Fetching page: "Vibe coding"
Information Source: https://en.wikipedia.org/wiki/Vibe_coding
Fetching page: "DeepSeek"
Information Source: https://en.wikipedia.org/wiki/DeepSeek
Information Sources:
     https://en.wikipedia.org/wiki/Large_language_model
     https://en.wikipedia.org/wiki/Vibe_coding
     https://en.wikipedia.org/wiki/DeepSeek


Here is what the search results look like:

In [7]:
from IPython.display import display, Markdown

for e in example:
  display(Markdown(e))

Based on the provided source, here's the relevant information about what LLMs are:

*   Vibe coding is an AI-dependent programming technique where a person describes a problem in a few sentences as a prompt to a large language model (LLM) tuned for coding.
*   The concept refers to a coding approach that relies on LLMs, allowing programmers to generate working code by providing natural language descriptions rather than manually writing it.
*   Karpathy described his approach as conversational, using voice commands while AI generates the actual code.
*   the capabilities of LLMs were such that humans would no longer need to learn specific programming languages to command computers.


Based on:
  https://en.wikipedia.org/wiki/Vibe_coding

Large language models (LLMs) are developed by DeepSeek, a Chinese artificial intelligence company. DeepSeek-R1, one of their models, provides responses comparable to other LLMs like OpenAI's GPT-4 and o1. The training cost for DeepSeek's LLMs is reported to be significantly lower than others. DeepSeek's models are described as "open weight."


Based on:
  https://en.wikipedia.org/wiki/DeepSeek

### Step 2: Pass the System Instructions and Tools to the Model

### Define the prompt

In order to have multiple supporting search queries to the user's original query, you will instruct the model to generate more such queries. This would help the engine to cover the asked question on comprehensive levels.

In [8]:
instructions = """
    You have access to the Wikipedia API which you will be using
    to answer a user's query. Your job is to generate a list of search queries which
    might answer a user's question. Be creative by using various key-phrases from
    the user's query. To generate variety of queries, ask questions which are
    related to the user's query that might help to find the answer. The more
    queries you generate the better are the odds of you finding the correct answer.
    Here is an example:

    user: Tell me about Cricket World cup 2023 winners.

    function_call: wikipedia_search([
        'What is the name of the team that won the Cricket World Cup 2023?',
        'Who was the captain of the Cricket World Cup 2023 winning team?',
        'Which country hosted the Cricket World Cup 2023?',
        'What was the venue of the Cricket World Cup 2023 final match?',
        'Cricket World cup 2023',
        'Who lifted the Cricket World Cup 2023 trophy?'
    ])

    The search function will return a list of article summaries, use these to
    answer the  user's question.
"""

When using the Python SDK, you can provide Python functions directly as tools. The SDK automatically converts the Python function to declarations, handles the function call execution and response cycle for you. The Python SDK then automatically:

- Detects function call responses from the model.
- Call the corresponding Python function in your code.
- Sends the function response back to the model.
- Returns the model's final text response.

To use this, define your function with type hints and a docstring as we did above, and then pass the function itself (not a JSON declaration) as a tool:

Note: This approach only handles annotations of `AllowedType = (int | float | bool | str | list['AllowedType'] | dict[str, AllowedType])`

In order to yield creative and a more random variety of questions, you could also utilize the model's temperature parameter. Values can range from [0.0,1.0], inclusive. A value closer to 1.0 will produce responses that are more varied and creative, while a value closer to 0.0 will typically result in more straightforward responses from the model.

In [9]:
from google.genai import types

config = types.GenerateContentConfig(
    tools=[wikipedia_search],
    system_instruction=instructions,
) # Pass the function itself

### Step 3: Enable Automatic Function Calling and Call the API (Python SDK Only)

Now start a new chat with default value of `automatic_function_calling`. Using the default `disable=None`, the `genai.ChatSession` will handle the back and forth required to call the function, and return the final response:

In [10]:
chat = client.chats.create(model=model, config=config, history=[])

query = "Explain how deep-sea life survives."

response = chat.send_message(query)

Searching for "deep sea life adaptations"
Related search terms: ['Deep sea', 'Deep-sea fish', 'Deep-sea gigantism']
Fetching page: "Deep sea"
Information Source: https://en.wikipedia.org/wiki/Deep_sea
Fetching page: "Deep-sea fish"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_fish
Deep-sea fish did not match with any page id, hence skipping.
Fetching page: "Deep-sea gigantism"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_gigantism
Searching for "deep sea food sources"
Related search terms: ['Deep-sea fish', 'Deep sea', 'Deep-sea community']
Fetching page: "Deep-sea community"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_community
Searching for "deep sea environment challenges"
Related search terms: ['Deep-sea community', 'Deepsea Challenger', 'Deep sea']
Fetching page: "Deepsea Challenger"
Information Source: https://en.wikipedia.org/wiki/Deepsea_Challenger
Searching for "deep sea organisms survival strategies"
Related search terms: ['List of l

In [11]:
display(Markdown(response.text))

Deep-sea life survives by adapting to the extreme conditions of low temperatures, darkness, and high pressure. Here's a breakdown:

*   **Adaptations to the Environment:** Organisms have developed jelly-like flesh for buoyancy, large eyes with rod cells for low light, and specialized proteins and metabolic systems to withstand hydrostatic pressure. Some fish also have a retroreflector behind the retina for more sensitive vision in low light.

*   **Food Sources:** Deep-sea creatures rely on marine snow (organic material falling from upper waters), scavenging, predation, and filtration. Some species at hydrothermal vents use chemosynthesis instead of dissolved organic matter, like the symbiotic relationship between tube worms and chemosynthetic bacteria.

*   **Deep-Sea Gigantism:** Some deep-sea animals tend to be larger than their shallower-water relatives, potentially due to colder temperatures, food scarcity, reduced predation pressure, and increased dissolved oxygen concentrations.

*   **Energy Conservation:** Some species conserve energy by retaining water in their gills for extended periods.

In extreme environments like brine pools, organisms have further adaptations to survive high salinity, lack of oxygen, and extreme temperatures, such as using alternative electron acceptors and forming symbiotic relationships with bacteria.


Check for additional citations:

In [12]:
response.candidates[0].citation_metadata or 'No citations found'

'No citations found'

### Step 4: Understand Chat History

That looks like it worked. You can go through the chat history to see the details of what was sent and received in the function calls:

In [14]:
import json
for content in chat.get_history():
    display(Markdown("### " + content.role + ":"))
    for part in content.parts:
        if part.text:
            display(Markdown(part.text))
        if part.function_call:
            print(part.function_call.name, json.dumps(part.function_call.args, indent=2))
        if part.function_response:
            for res in part.function_response.response['result']:
                display(Markdown(res))
    print("-" * 80)

### user:

Explain how deep-sea life survives.

--------------------------------------------------------------------------------


### model:

wikipedia_search {
  "search_queries": [
    "deep sea life adaptations",
    "deep sea food sources",
    "deep sea environment challenges",
    "deep sea organisms survival strategies",
    "hydrothermal vent ecosystems",
    "deep sea pressure adaptations",
    "deep sea bioluminescence",
    "deep sea chemosynthesis",
    "How do deep-sea creatures get energy?",
    "What are the challenges of living in the deep sea?",
    "What are the adaptations of deep-sea animals?"
  ]
}
--------------------------------------------------------------------------------


...

This document primarily focuses on the surveys and bathymetry (depth measurements) of the Challenger Deep. It does not explicitly list challenges of living in the deep sea. However, some implicit challenges can be inferred from the text:

*   **Extreme Pressure:** The text details the equipment and engineering required to withstand the immense pressure at the Challenger Deep, implying that this is a major obstacle for any life. Vessels needed thick steel walls to resist being crushed.
*   **Darkness:** The observation by the Trieste crew that the floodlight they used was "the first real light" to enter that realm points to the challenge of living in complete darkness.
*   **Limited Nutrient Availability/Specific Diet:** The description of organisms that live at this depth (foraminifera, amphipods) and their diet (diatomaceous ooze, bottom-feeders) suggests the specialized adaptations required to survive in an environment with limited and unique food sources. The challenge of growing calcium-based shells.
*   **Cold Temperatures:** Mentions of temperature readings (2.6°C) demonstrate the challenges of living in extremely cold environment.
*   **Scarcity of life** Camera recordings revealed only holothurians, White polychaetes (bristle worms), tube worms, and other biological species.


Based on:
  https://en.wikipedia.org/wiki/Challenger_Deep

--------------------------------------------------------------------------------


### model:

Deep-sea life survives by adapting to the extreme conditions of low temperatures, darkness, and high pressure. Here's a breakdown:

*   **Adaptations to the Environment:** Organisms have developed jelly-like flesh for buoyancy, large eyes with rod cells for low light, and specialized proteins and metabolic systems to withstand hydrostatic pressure. Some fish also have a retroreflector behind the retina for more sensitive vision in low light.

*   **Food Sources:** Deep-sea creatures rely on marine snow (organic material falling from upper waters), scavenging, predation, and filtration. Some species at hydrothermal vents use chemosynthesis instead of dissolved organic matter, like the symbiotic relationship between tube worms and chemosynthetic bacteria.

*   **Deep-Sea Gigantism:** Some deep-sea animals tend to be larger than their shallower-water relatives, potentially due to colder temperatures, food scarcity, reduced predation pressure, and increased dissolved oxygen concentrations.

*   **Energy Conservation:** Some species conserve energy by retaining water in their gills for extended periods.

In extreme environments like brine pools, organisms have further adaptations to survive high salinity, lack of oxygen, and extreme temperatures, such as using alternative electron acceptors and forming symbiotic relationships with bacteria.


--------------------------------------------------------------------------------


In the chat history you can see all 4 steps:

1. User: Asks the question about the total number of mittens.
2. Model: Determines that the `wikipedia_search` is helpful and sends a FunctionCall request to the user.
3. User: The Chat session automatically executes the function (due to `_automatic_function_calling` is enabled by default) and sends back a FunctionResponse with the calculated result.
4. Model: Uses the function's output to formulate the final answer and presents it as a text response.

## [Optional] Manually Execute the Function Call

If you want to understand what happened behind the scenes, this section executes the `FunctionCall` manually to demonstrate.

In [15]:
config  = {
        "tools": [wikipedia_search],
        "automatic_function_calling": {"disable": True}, # for manual execution
    }

chat    = client.chats.create(model=model, config=config, history=[])
response  = chat.send_message(query)

Initially the model returns a `FunctionCall`:

In [16]:
fc = response.candidates[0].content.parts[0].function_call
print(json.dumps(fc.args, indent=2))

{
  "search_queries": [
    "deep sea life survival"
  ]
}


Call the function with generated arguments to get the results.

In [17]:
if fc.name == "wikipedia_search":
    summaries = wikipedia_search(**fc.args)
    display(Markdown("\n ### Function execution result:"))
    for text in summaries:
        display(Markdown(text))

Searching for "deep sea life survival"
Related search terms: ['Deep sea mining', 'Stranded Deep', 'Deep-sea community']
Fetching page: "Deep sea mining"
Information Source: https://en.wikipedia.org/wiki/Deep_sea_mining
Fetching page: "Stranded Deep"
Information Source: https://en.wikipedia.org/wiki/Stranded_Deep
Fetching page: "Deep-sea community"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_community
Deep-sea community did not match with any page id, hence skipping.
Information Sources:
     https://en.wikipedia.org/wiki/Deep_sea_mining
     https://en.wikipedia.org/wiki/Stranded_Deep
     https://en.wikipedia.org/wiki/Deep-sea_community



 ### Function execution result:

The provided text does not contain information about how deep sea life survives. It primarily focuses on deep sea mining, its processes, environmental impacts, and related legal and economic aspects.


Based on:
  https://en.wikipedia.org/wiki/Deep_sea_mining

*   Stranded Deep takes place in the Pacific Ocean.
*   A plane crash survivor explores Pacific islands, reefs, and bottomless ocean trenches.
*   Players need to search for and develop the means to survive.
*   Players need to manage their vitals of health, hunger, thirst, and sleep.
*   Thirst is an important component of survival.
*   Hydration sources are scarce, with coconut water being the most common.
*   Food can be obtained from coconuts, fishing, and rations.
*   The tropical ocean is full of life forms, including small fish, sea turtles, rays, lionfish, sea snakes, crown-of-thorns starfish, swordfish, whales, and sharks.
*   There are different types of sharks, some harmless, others life-threatening like great whites, hammerhead shark, and goblin shark.
*   Sharks can attack the player or flip their raft.
*   There are three mythical sea creatures: a megalodon shark, a giant squid named Lusca, and a giant eel named Abaia.


Based on:
  https://en.wikipedia.org/wiki/Stranded_Deep

Now send the `summaries` to the model.

In [18]:
# Create a function response part
function_response_part = types.Part.from_function_response(
    name=fc.name,
    response={"result": summaries},
)

response = chat.send_message(function_response_part)

display(Markdown(response.text))

I'm sorry, but I do not have the information to explain how deep-sea life survives. The search results I have are about deep-sea mining and the video game Stranded Deep, which contains some information about sea creatures, but not about their survival in the deep sea.

## Re-ranking the search results

Helper function to embed the content:

In [19]:
# @title Helper function to embed the content
from tqdm.auto import tqdm
from google.genai import types

tqdm.pandas()

from google.api_core import retry
import numpy as np

def make_embed_text_fn(model):

    @retry.Retry(timeout=300.0)
    def embed_fn(texts: list[str]) -> list[list[float]]:
        # Set the task_type to SEMANTIC_SIMILARITY and embed the batch of texts
        embeddings = client.models.embed_content(
            model=model,
            contents=texts,
            config=types.EmbedContentConfig(task_type="SEMANTIC_SIMILARITY"),
        ).embeddings
        return np.array([embedding.values for embedding in embeddings])

    return embed_fn


def create_embeddings(content: list[str]) -> np.ndarray:
    MODEL_ID = "embedding-001" # @param ["embedding-001", "text-embedding-004","gemini-embedding-exp-03-07"] {"allow-input":true, isTemplate: true}
    model = f"models/{MODEL_ID}"
    embed_fn = make_embed_text_fn(model)

    batch_size = 100  # at most 100 requests can be in one batch
    all_embeddings = []

    # Loop over the texts in chunks of batch_size
    for i in tqdm(range(0, len(content), batch_size)):
        batch = content[i:i + batch_size]
        embeddings = embed_fn(batch)
        all_embeddings.extend(embeddings)

    return np.array(all_embeddings).reshape(len(all_embeddings), -1)

Please refer to the [embeddings guide](https://ai.google.dev/docs/embeddings_guide) for more information on embeddings.

Your next step is to define functions that you can use to calculate similarity scores between two embedding vectors. These scores will help you decide which embedding vector is the most relevant vector to the user's query.


You will now implement cosine similarity as your metric. Here returned embedding vectors will be of unit length and so their L1 norm (`np.linalg.norm()`) will be ~1. Hence, calculating cosine similarity is esentially same as calculating their dot product score.

In [20]:
def dot_product(a: np.ndarray, b: np.ndarray):
  return (a @ b.T)

### Similarity with user's query

Now it's time to find the most relevant search result returned by the Wikipedia API.

Use Gemini API to get embeddings for user's query and search results.

In [21]:
search_res = create_embeddings(summaries)
embedded_query = create_embeddings([query])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Calculate similarity score:

In [22]:
sim_value = dot_product(search_res, embedded_query)
sim_value

array([[0.81370797],
       [0.65965196]])

using `np.argmax` best candidate is selected.

**Users's Input:** Explain how deep-sea life survives.

**Answer:**

In [23]:
display(Markdown(summaries[np.argmax(sim_value)]))

The provided text does not contain information about how deep sea life survives. It primarily focuses on deep sea mining, its processes, environmental impacts, and related legal and economic aspects.


Based on:
  https://en.wikipedia.org/wiki/Deep_sea_mining

### Similarity with Hypothetical Document Embeddings (HyDE)

Drawing inspiration from [Gao et al](https://arxiv.org/abs/2212.10496) the objective here is to generate a template answer to the user's query using `gemini-2.0-flash`'s internal knowledge. This hypothetical answer will serve as a baseline to calculate relevance of all the search results.

In [24]:
res = client.models.generate_content(
    model=model,
    contents=f"""
        Generate a hypothetical answer
        to the user's query by using your own knowledge. Assume that you know everything
        about the said topic. Do not use factual information, instead use placeholders
        to complete your answer. Your answer should feel like it has been written by a human.

        query: {query}
    """,
)

display(Markdown(res.text))

Okay, so imagine you're trying to survive in a place where the pressure is like having [Placeholder: Ridiculous Number] elephants standing on your head, and it's darker than [Placeholder: Something Incredibly Dark, Like the Inside of a Black Hole] all the time. Plus, food is scarcer than [Placeholder: Something Extremely Rare, like Unicorn Tears]. That's basically the deep sea!

So how do these crazy creatures manage? Well, it's all about adaptation, baby!

First off, dealing with that crushing pressure. Many deep-sea creatures have bodies that are mostly water - we're talking like [Placeholder: Percentage]%! This helps them equalize the pressure. They also often lack swim bladders, the gas-filled sacs that help fish float at shallower depths, because those would just get squished. Think of it like having a super flexible, adaptable jelly-like body, but not actually jelly because that's just a simple example for us surface dwellers.

Then there's the darkness. Photosynthesis? Forget about it! No sunlight means no plants. So, the food chain starts with "marine snow" – basically, organic debris raining down from above. Some creatures are masters of scavenging, hoovering up this stuff like they're cleaning up after a [Placeholder: Messy Creature]. Others have developed amazing bioluminescence. They can produce their own light to lure prey, communicate with each other (maybe even have little underwater dance parties!), or even startle predators with a flash of light like a tiny, deep-sea [Placeholder: Creature With a Flashlight].

And finally, the food scarcity. Strategies here are all over the place. Some are ambush predators, sitting perfectly still and waiting for an unsuspecting [Placeholder: Deep-Sea Prey] to wander by. Others have incredibly slow metabolisms, meaning they barely need to eat anything at all. They're basically like the deep-sea equivalent of a [Placeholder: Animal Known for Slow Metabolism]. They can conserve energy and survive for extended periods between meals.

It's a tough life down there, but the creatures that have adapted to it are absolutely fascinating. They’re masters of survival in one of the most extreme environments on our planet! And trust me, there's still SO much we don't know about them - it's truly a whole other world down there.


Use Gemini API to get embeddings for the baseline answer and compare them with search results

In [25]:
hypothetical_ans = create_embeddings([res.text])

  0%|          | 0/1 [00:00<?, ?it/s]

Calculate similarity scores to rank the search results

In [26]:
sim_value = dot_product(search_res, hypothetical_ans)
sim_value

array([[0.726802  ],
       [0.72210996]])

using `np.argmax` best candidate is selected.

**Users's Input:** Explain how deep-sea life survives.

**Answer:**

In [27]:
display(Markdown(summaries[np.argmax(sim_value)]))

The provided text does not contain information about how deep sea life survives. It primarily focuses on deep sea mining, its processes, environmental impacts, and related legal and economic aspects.


Based on:
  https://en.wikipedia.org/wiki/Deep_sea_mining

You have now created a search re-ranking engine using embeddings!

## Next steps

I hope you found this example helpful! Check out more examples in the [Cookbook](https://github.com/google-gemini/cookbook) to learn more.